In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
test = pd.read_csv('../input/titanic/test.csv')
train = pd.read_csv('../input/titanic/train.csv')

In [3]:
df_test = pd.DataFrame(test)
df = pd.DataFrame(train)

In [4]:
df.head()

In [5]:
df["FamilyS"] = df["SibSp"] + df["Parch"] + 1
df_test["FamilyS"] = df_test["SibSp"] + df_test["Parch"] + 1

In [6]:
df.isnull().sum()

In [7]:
df['Age'].fillna(df['Age'].median(), inplace = True)
df_test['Age'].fillna(df['Age'].median(), inplace = True)
df['Cabin'].fillna(df['Cabin'].mode()[0], inplace = True)
df_test['Cabin'].fillna(df['Cabin'].mode()[0], inplace = True)

In [12]:
dummies = pd.get_dummies(df[['Sex','Embarked']])

In [13]:
dummies_test = pd.get_dummies(df[['Sex','Embarked']])

In [16]:
df = df.join(dummies)
df_test = df_test.join(dummies_test)

In [17]:
df.head()

In [18]:
df_test.head()

In [19]:
df = df.drop(['Name','Ticket','PassengerId','Cabin','SibSp','Parch','Sex','Embarked'], axis = 1)
df_test = df_test.drop(['Name','Ticket','PassengerId','Cabin','SibSp','Parch','Sex','Embarked'], axis = 1)

In [20]:
df_test.isnull().sum()

In [21]:
df_test['Fare'].fillna(df_test['Fare'].mean(),inplace = True)

In [23]:
df.head()

In [22]:
df_test.head()

In [24]:
y_train = df['Survived'].values
X_train = df.drop(['Survived'], axis = 1).values
X_test = df_test.values

In [25]:
y_train.shape, X_train.shape, X_test.shape

In [26]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [27]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [28]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [36]:
classifier = Sequential()
classifier.add(Dense(activation="relu", input_dim=9, units=11, kernel_initializer="uniform"))
classifier.add(Dense(activation="relu", units=11, kernel_initializer="uniform"))
classifier.add(Dropout(0.5))
classifier.add(Dense(activation="relu", units=11, kernel_initializer="uniform"))
classifier.add(Dropout(0.5))
classifier.add(Dense(activation="relu", units=5, kernel_initializer="uniform"))
classifier.add(Dense(activation="sigmoid", units=1, kernel_initializer="uniform"))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.summary()

In [38]:
history=classifier.fit(X_train, y_train, batch_size = 4, epochs = 200,
    validation_split=0.1,verbose = 1,shuffle=True)

In [44]:
Y_pred = classifier.predict(df_test)

In [45]:

Y_pred=Y_pred.round()
Y_pred

In [47]:
def write_prediction(prediction, name):
    PassengerId = np.array(test['PassengerId']).astype(int)
    solution = pd.DataFrame(prediction, PassengerId, columns = ['Survived'])
    solution.to_csv(name, index_label = ['PassengerId'])

In [48]:
write_prediction(Y_pred, "My_output.csv")